# Attack
In this notebook, we demonstrate the proposed attack algorithm by conducting pre-image attacks on templates enrolled in `2. LSHS`. For comparison, we also implemented a previous genetic algorithm-based attack.

In [ ]:
import torch
import torch.nn.functional as F

from feats.utils_feat import feat_ext, load_bin
from fr_utils import get_backbone_v2
from INV import get_nbnet
from preset import get_LSH_preset
from eval_attack import eval_ASR, eval_ASR_GEN, Recover_pre, Recover_pre_GEN

In [ ]:
# Settings
device = "" # device setting "cpu" or "cuda"
target_dir = "" # input your pre-trained target backbone model's directory
nbnet_dir = "" # input your pre-trainde NbNet model's directory
test_dir = "" # input your dataset(LFW)'s directory
tmp_dir = "" # directory where you've already saved LSH templates

### Get Preset LSHs

In [ ]:
LSHs = get_LSH_preset(device)

### Load taget model & Extract Feature

In [ ]:
r50_glint_arc = get_backbone_v2(("r50", target_dir, False), device) # If supported by get_backbone_v2, you can change "r50" to  other models.
nbnet = get_nbnet(nbnet_dir, device)
lfw = load_bin(test_dir + '/lfw.bin', (112,112))
lfw_feat = feat_ext(lfw, r50_glint_arc, 512, device)

### Attack Setting 
##### atk_[LSH] = (initailization, loss(mean/sum), loss(absolute/pow))
##### thx_[LSH] = (threshold)

In [ ]:
atk_configs = dict()
atk_configs["BH"] = (True, "s", "abs")
atk_configs["GRP"] = (True, "s", "abs")
atk_configs["URP"] = (False, "s", "abs")
atk_configs["DIoM"] = (True, "s", "abs")
atk_configs["IMM"] = (True, "s", "abs")
atk_configs["ABH"] = (True, "s", "abs")
atk_configs["Jiang"] = (True, "s", "pow")

thx_configs = dict()
thx_configs["BH"] = 293
thx_configs["GRP"] = 43
thx_configs["URP"] = 23
thx_configs["DIoM"] = 291
thx_configs["IMM"] = 90
thx_configs["ABH"] = 1
thx_configs["Jiang"] = 181
thx_configs["backbone"] = 0.2533

### Recovery pre_image

**NOTE** This reproduces the Table 4 (Purity Analysis).

In [ ]:
pre_out_dir = "" # directory where you want to save the recovered pre-image from LSH templates by proposed method
_ = Recover_pre(LSHs, lfw_feat, atk_configs, device, tmp_dir=tmp_dir, out_dir=pre_out_dir)

pre_gen_out_dir = "" # directory where you want to save the recovered pre-image from LSH templates by genetic method
_ = Recover_pre_GEN(LSHs, lfw_feat, atk_configs, device, tmp_dir=tmp_dir, out_dir=pre_gen_out_dir)

### Proposed Attack - ASR1, ASR2

**NOTE** This reproduces the remaining rows in Table 3.

In [ ]:
_ = eval_ASR(LSHs, thx_configs, lfw_feat, r50_glint_arc, nbnet, device)

### Genetic Attack - ASR1, ASR2

**NOTE** This reproduces the remaining rows in Table 3.

In [ ]:
_ = eval_ASR_GEN(LSHs, thx_configs, lfw_feat, r50_glint_arc, nbnet, device)